In [35]:
import pandas as pd
import numpy as np
import datetime as dt

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.options.plotting.backend = "plotly"

In [5]:
download_path = '/Users/dhan/Codes/notebook/20230915 tinysoft 天软/'

In [37]:
import pyTSL
from datetime import datetime
import pandas as pd


args_dict = {
    'date': "datetimetostr(['date']) as 'time'",
    'StockID': "['StockID'] as 'symbol_id'",
    'StockName': "['StockName'] as 'symbol_name'",
    'price': "['price'] as 'last'",
    'vol': "['vol'] as 'trade_volume'",
    'amount': "['amount'] as 'trade_amount'",
    'cjbs': "['cjbs'] as 'open_interest_change'",
    'bc1': "['bc1'] as 'bv1'",
    'buy1': "['buy1'] as 'bp1'",
    'sale1': "['sale1'] as 'ap1'",
    'sc1': "['sc1'] as 'av1'",
    'sectional yclose': "['sectional_yclose'] as 'yesterday_close'",
    'sectional_open': "['sectional_open'] as 'today_open'",
    'sectional_high': "['sectional_high'] as 'today_high'",
    'sectional_low': "['sectional_low'] as 'today_low'",
    'sectional_vol': "['sectional_vol'] as 'volume'",
    'sectional_amount': "['sectional_amount'] as 'amount'",
    'sectional_cjbs': "['sectional_cjbs'] as 'open_interest'",
    'open': "['open']  as 'open'",                          # for k only
    'high': "['high']  as 'high'",                          # for k only
    'low': "['low']  as 'low'",                             # for k only
    'yclose': "['yclose']  as 'close'",                     # for k only
    'Syl2': "['Syl2']  as 'yesterday_settlementprice'",      # for k only
    '合约代码': 703001,
    '变动日': 703002,
    '交易代码':  703003,
    '交割年份':	 703004,
    '交割月份':	 703005,
    '交易品种':	 703006,
    '合约乘数':	 703007,
    '合约乘数单位': 703008,
    '报价单位': 703009,
    '最小变动价位': 703010,
    '每日价格最大波动下限(%)': 703011,
    '每日价格最大波动上限(%)':	 703012,
    '最后交易日参照标准': 703013,
    '最后交易日相对参照标准偏移月份': 703014,
    '最后交易日类别': 703015,
    '最后交易日相对最后交易日所在月份偏移天数': 703016,
    '最后交易日是否假日顺延': 703017,
    '最后交易日': 703018,
    '最后交割日参照标准': 703019,
    '最后交割日相对参照标准偏移月份': 703020,
    '最后交割日类别': 703021,
    '最后交割日相对最后交割日所在月份偏移天数': 703022,
    '最后交割日是否假日顺延': 703023,
    '最后交割日': 703024,
    '最低交易保证金(%)': 703025,
    '交割方式': 703026,
    '上市地': 703027,
    '期货类别': 703028,
    '商品期货类别': 703029,
    '基准代码': 703030
}

tick_args = [
    'date', 'StockID', 'StockName', 'price', 'vol', 'amount', 'cjbs', 'bc1', 'buy1', 'sale1', 'sc1', 'sectional yclose',
    'sectional_open', 'sectional_high', 'sectional_low', 'sectional_vol', 'sectional_amount', 'sectional_cjbs'
]

k_args = [
    'date', 'StockID', 'StockName', 'price', 'vol', 'amount', 
    'cjbs', 'open', 'high', 'low', 'yclose', 'Syl2',
    'sectional yclose', 'sectional_open', 'sectional_high', 'sectional_low', 'sectional_vol', 'sectional_amount', 'sectional_cjbs'
]

future_args = [ 
    '合约代码',
    '变动日',	
    '交易代码',	
    '交割年份',	
    '交割月份',	
    '交易品种',	
    '合约乘数',	
    '合约乘数单位',
    '报价单位',
    '最小变动价位',
    '每日价格最大波动下限(%)',
    '每日价格最大波动上限(%)',
    '最后交易日参照标准',
    '最后交易日相对参照标准偏移月份',
    '最后交易日类别',
    '最后交易日相对最后交易日所在月份偏移天数',
    '最后交易日是否假日顺延',
    '最后交易日',
    '最后交割日参照标准',
    '最后交割日相对参照标准偏移月份',
    '最后交割日类别',
    '最后交割日相对最后交割日所在月份偏移天数',
    '最后交割日是否假日顺延',
    '最后交割日',
    '最低交易保证金(%)',
    '交割方式',
    '上市地',
    '期货类别',
    '商品期货类别',
    '基准代码'
]


# 需要根据所需路径进行更改
path = {
    'data_save': download_path
}

def TS_INIT():
    c = pyTSL.Client("hit", "20230427", "tsl.tinysoft.com.cn", 443)
    c.login()
    return c


def write_file(df, name):
    output_path = path['data_save'] + name
    df= pd.DataFrame(df)
    df.to_csv(output_path)
    print(df.shape, output_path)
    return 1
pyTSL.register_proc('write_file', write_file)


def code_get_bk_list(bk_name=''):
    if bk_name == '':
        code = '''
            return getBKList2();
            '''
    else:
        # 层级结构不好用，还没找到原因
        code = '''
            return getBKList2('%s');
            ''' % bk_name
    return code


def code_get_bk(bk_name):
    code = '''
        return getbk('%s');
        ''' % bk_name
    return code

def code_OP_GetOptionChain(pz, Endt):
    """
    输入期货名称和截止日期, 返回TSL语言代码, 可获得期权列表
    """
    code = '''
        PZ:= '%s';
        Endt:= %s;
        data:= OP_GetOptionChain(PZ, Endt);
        
        rdo2 write_file(data, datetimetostr(Endt)+'_'+PZ+'_baseInfo.csv');
        return Over;
        ''' % (pz, Endt)
    return code


def code_get_data(begT, endT, args, bk_name = '', stock_name = '', time_cycle = ''):
    """
    获取并返回TSL语言代码, 执行该代码可直接下载某板块中的所有期货或某个期货的数据(k线或tick线)
    :param begT: 数据起始时间(包括该日数据)
    :param endT: 数据终止时间(包括该日数据)
    :param bk_name: 板块名称(可为空), 若板块名称不为空，下载该板块下的所有股票， 
    :param stock_name: 股票名称(可为空), 若股票名称不为空且板块名称为空，则下载该股票的内容
    :param time_cycle: 行情频率(可为空), 所需数据为tick线时, 设定为空, 所需数据为k线时, 设置为所需的频率
    """
    arg_string = ''
    for arg in args:
        arg_string += args_dict[arg]
        if arg != args[-1]:
            arg_string += ','

    if time_cycle != '':
        set_time_cycle_string = 'setsysparam(pn_cycle(), %s);'% time_cycle
        table = 'markettable'
        type_string = 'k'
    else:
        set_time_cycle_string = ''
        table = 'tradetable'
        type_string = 'tick'

    if bk_name != '':
        code = '''
            begT := %s;
            endT := %s;
            bkArr:= getbk('%s');
            
            Tarr := MarketTradeDayQk(begT, endT);        
            for t_index:=0 to length(Tarr)-1 do
            begin
                
                %s
                bk :=   
                    sselect thisrow 
                    from bkArr
                    where spec(FuturesIsTrade(Tarr[t_index]), thisrow) = 1
                    end;
                    
                for i:=0 to length(bk)-1 do
                begin
                    SetSysParam(pn_Stock(),bk[i]);
                    SetSysParam(pn_date(), Tarr[t_index]);
                    if not istradeday(Tarr[t_index]) then continue;
                    BegTT := ref(sp_time(),1)+18/24;
                    EndTT := Tarr[t_index]+18/24;

                    data := select
                            %s
                        from %s
                        datekey BegTT to EndTT
                        of bk[i]
                        end;
                    FileName := bk[i];

                    rdo2 write_file(data, datetimetostr(Tarr[t_index])+'_'+FileName+'_%s'+'.csv');
                end;
            end;
            return 'Over';
        ''' % (begT, endT, bk_name, set_time_cycle_string, arg_string, table, type_string)
    else:
        code = '''
            begT := %s;
            endT := %s;
            bk   := '%s';
            SetSysParam(PN_Stock(),bk);
            Tarr := MarketTradeDayQk(begT, endT);                    
                                    
            for t_index:=0 to length(Tarr)-1 do
            begin
                SetSysParam(pn_date(), Tarr[t_index]);
                if not istradeday(Tarr[t_index]) then continue;
                BegTT := ref(sp_time(),1)+18/24;
                EndTT := Tarr[t_index]+18/24;
                echo sp_time();
                %s
                data := select
                    %s
                    from %s
                    datekey BegTT to EndTT
                    of bk
                    end;

                
                FileName := bk;
                rdo2 write_file(data, datetimetostr(Tarr[t_index])+'_'+FileName+'_%s'+'.csv');
            end;
            return 'Over';
        ''' % (begT, endT, stock_name, set_time_cycle_string, arg_string, table, type_string)
    return code

def code_get_future_baseInfo(future_name, args):
    args_string = 'array('
    for arg in args:
        args_string += 'base(%s)' % args_dict[arg]
        if arg != args[-1]:
            args_string += ','
        else:
            args_string += ')'

    
    code = '''
            SetSysParam(Pn_Stock(), "%s");
            return %s;
        ''' % (future_name, args_string)
    return code


def get_futureBK_baseInfo(c, bk_name, future_args):
    output_data = {arg:[] for arg in future_args}

    code = code_get_bk(bk_name)
    for future_name in c.exec(code).value():
        code = code_get_future_baseInfo(future_name, future_args)
        data = c.exec(code).value()
        for index, arg in enumerate(future_args):
            output_data[arg].append(data[index])
    df = pd.DataFrame.from_dict(output_data)
    df.to_csv(path['data_save']+'future_baseInfo_'+bk_name+'.csv', index=False)
    print('('+str(df.shape[0])+', '+str(df.shape[1])+') '+ path['data_save']+'future_baseInfo_'+bk_name+'.csv')


def get_bk_values(c, name):
    code = code_get_bk(name)
    print(code)
    print(c.exec(code).value())
    
    
def get_FuturesAllPZCode(c):
    code = '''
         return FuturesAllPZCode();
        '''
    return pd.DataFrame(c.exec(code).value())

def get_OP_GetUnderlyingSecurity(c):
    code = '''
         return OP_GetUnderlyingSecurity();
        '''
    return pd.DataFrame(c.exec(code).value())


In [10]:
c = TS_INIT()

In [8]:
code = code_OP_GetOptionChain('zn2301', '20231010T')
print(code)
r = c.exec(code)


        PZ:= 'zn2301';
        Endt:= 20231010T;
        data:= OP_GetOptionChain(PZ, Endt);
        
        rdo2 write_file(data, datetimetostr(Endt)+'_'+PZ+'_baseInfo.csv');
        return Over;
        


In [16]:
get_futureBK_baseInfo(c, '螺纹钢', future_args)

(12, 30) /Users/dhan/Codes/notebook/20230915 tinysoft 天软/future_baseInfo_螺纹钢.csv


In [25]:
get_bk_values(c, '上市期货')


        return getbk('上市期货');
        
['AP2310', 'AP2311', 'AP2312', 'AP2401', 'AP2403', 'AP2404', 'AP2405', 'CF2311', 'CF2401', 'CF2403', 'CF2405', 'CF2407', 'CF2409', 'CJ2312', 'CJ2401', 'CJ2403', 'CJ2405', 'CJ2407', 'CJ2409', 'CY2310', 'CY2311', 'CY2312', 'CY2401', 'CY2402', 'CY2403', 'CY2404', 'CY2405', 'CY2406', 'CY2407', 'CY2408', 'CY2409', 'FG2310', 'FG2311', 'FG2312', 'FG2401', 'FG2402', 'FG2403', 'FG2404', 'FG2405', 'FG2406', 'FG2407', 'FG2408', 'FG2409', 'IC2310', 'IC2311', 'IC2312', 'IC2403', 'IF2310', 'IF2311', 'IF2312', 'IF2403', 'IH2310', 'IH2311', 'IH2312', 'IH2403', 'IM2310', 'IM2311', 'IM2312', 'IM2403', 'JR2311', 'JR2401', 'JR2403', 'JR2405', 'JR2407', 'JR2409', 'LR2311', 'LR2401', 'LR2403', 'LR2405', 'LR2407', 'LR2409', 'MA2310', 'MA2311', 'MA2312', 'MA2401', 'MA2402', 'MA2403', 'MA2404', 'MA2405', 'MA2406', 'MA2407', 'MA2408', 'MA2409', 'OI2311', 'OI2401', 'OI2403', 'OI2405', 'OI2407', 'OI2409', 'PF2310', 'PF2311', 'PF2312', 'PF2401', 'PF2402', 'PF2403', 'PF2404',

In [36]:
get_FuturesAllPZCode(c)

,品种代码,变动日,品种名称,主力代码,指数线代码,连续代码,连一代码,连二代码,连三代码,连四代码
0,IC,20150416,中证500指数,IC00,ICInd,IC01,IC01,IC02,IC03,IC04
1,IF,20100416,沪深300指数,IF00,IFInd,IF01,IF01,IF02,IF03,IF04
2,IH,20150416,上证50指数,IH00,IHInd,IH01,IH01,IH02,IH03,IH04
3,IM,20220722,中证1000指数,IM00,IMInd,IM01,IM01,IM02,IM03,IM04
4,A,20020315,黄大豆1号,ZL000014,QI000014,LXA00,LXA01,LXA02,LXA03,LXA04
5,AG,20120510,白银,ZL000028,QI000028,LXAG00,LXAG01,LXAG02,LXAG03,LXAG04
6,AL,19920528,铝,ZL000001,QI000001,LXAL00,LXAL01,LXAL02,LXAL03,LXAL04
7,AO,20230619,氧化铝,ZL000067,QI000067,LXAO00,LXAO01,LXAO02,LXAO03,LXAO04
8,AP,20171222,苹果,ZL000049,QI000049,LXAP00,LXAP01,LXAP02,LXAP03,LXAP04
9,AU,20080109,黄金,ZL000002,QI000002,LXAU00,LXAU01,LXAU02,LXAU03,LXAU04


In [38]:
get_OP_GetUnderlyingSecurity(c)

,0
0,A
1,AG
2,AL
3,AU
4,B
5,BR
6,C
7,CF
8,CU
9,EB


In [54]:
code = code_OP_GetOptionChain('A', '20231010T')
print(code)
r = c.exec(code)


        PZ:= 'A';
        Endt:= 20231010T;
        data:= OP_GetOptionChain(PZ, Endt);
        
        rdo2 write_file(data, datetimetostr(Endt)+'_'+PZ+'_baseInfo.csv');
        return Over;
        
(294, 42) /Users/dhan/Codes/notebook/20230915 tinysoft 天软/2023-10-10_A_baseInfo.csv


In [55]:
get_bk_values(c, '上市期权;退市期权')


        return getbk('上市期权;退市期权');
        
['CF001C10800', 'CF001C11000', 'CF001C11200', 'CF001C11400', 'CF001C11600', 'CF001C11800', 'CF001C12000', 'CF001C12200', 'CF001C12400', 'CF001C12600', 'CF001C12800', 'CF001C13000', 'CF001C13200', 'CF001C13400', 'CF001C13600', 'CF001C13800', 'CF001C14000', 'CF001C14200', 'CF001C14400', 'CF001C14600', 'CF001C14800', 'CF001C15000', 'CF001C15200', 'CF001C15400', 'CF001C15600', 'CF001C15800', 'CF001C16000', 'CF001C16200', 'CF001C16400', 'CF001C16600', 'CF001C16800', 'CF001C17000', 'CF001C17200', 'CF001C17400', 'CF001C17600', 'CF001C17800', 'CF001P10800', 'CF001P11000', 'CF001P11200', 'CF001P11400', 'CF001P11600', 'CF001P11800', 'CF001P12000', 'CF001P12200', 'CF001P12400', 'CF001P12600', 'CF001P12800', 'CF001P13000', 'CF001P13200', 'CF001P13400', 'CF001P13600', 'CF001P13800', 'CF001P14000', 'CF001P14200', 'CF001P14400', 'CF001P14600', 'CF001P14800', 'CF001P15000', 'CF001P15200', 'CF001P15400', 'CF001P15600', 'CF001P15800', 'CF001P16000', 'CF001P16

期货
- 根据 get_FuturesAllPZCode 获得所有期货品种表table_1，存起来
- 使用 get_bk_values(c, '上市期货;退市期货') 获得所有期货合约名称存在table_2， 存起来
- 使用 regex 把期货合约名称 table_2 分类到品种代码上 table_1, 得到 AU: [au1901, ..., au2401]
- 使用 get_futureBK_baseInfo（目前你的function只能获得上市期货的info，没有退市的） 获得所有table_2里面的合约信息 得到table_3, 存起来
- 最后下载数据，文件夹层级为 品种代码（例如IC) - 日期 - 所有当日交易期货合约的行情数据

期权
- 根据 get_OP_GetUnderlyingSecurity 获得所有期权的标的物列表 table_1
- 使用 code_OP_GetOptionChain 可以得到某日某标的物（table 1的数据）的所有期权合约的详细信息，存起来table2
- 根据table_2, 可以下载里面每一个合约在那一天的行情数据
- 最后下载数据，文件夹层级为 品种代码（例如IC) - 日期 - 所有当日交易期权合约的行情数据